# LSTM长短期记忆

主要由四个函数构造的记忆细胞函数共同实现长短期记忆
- 记忆细胞
    - 遗忘门$F_t = \sigma(X_tW_{xf} + H_{t-1}W_{hf} + b_f)$
    - 输入门$I_t = \sigma(X_tW_{xi} + H_{t-1}W_{hi} + b_i)$
    - 候选记忆细胞$\tilde{C_t} = \tanh(X_tW_{xc} + H_{t-1}W_{hc} + b_c)$
    - 输出门$O_t = \sigma(X_tW_{xo} + H_{t-1}W_{ho} + b_o)$

通过以下的两个公式，上面四个函数被组装起来：
$$C_t = F_t \odot C_{t-1} + I_t \odot \tilde{C_t}. \tag{1}$$
$$H_t = O_t \odot \tanh(C_t).  \tag{2}$$

## 公式分析

分析以上的两个公式，可以发现：
- 当前记忆细胞通过函数，协调了**当前时间点输入的数据**和**上一时间点的输入数据**两者对隐藏层输入的影响
- 最后的隐藏层函数，则又对记忆细胞的记忆进行选择性保留（使用按位乘法）

通过两层次的协调，在经过训练后对于**当前层数据**和**过去层数据**的信息利用更加有复杂的拟合

## 实验 

In [1]:
# 数据导入和包导入
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = d2l.load_data_jay_lyrics()

In [3]:
# 初始化参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nd.zeros(num_hiddens, ctx=ctx))
    W_xi, W_hi, b_i = _three() # 输⼊⻔参数
    W_xf, W_hf, b_f = _three() # 遗忘⻔参数
    W_xo, W_ho, b_o = _three() # 输出⻔参数
    W_xc, W_hc, b_c = _three() # 候选记忆细胞参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc,
    b_c, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [4]:
# 定义模型
 def init_lstm_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),
            nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx))

In [5]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c,
    W_hq, b_q] = params
    
    (H, C) = state
    outputs = []
    for X in inputs:
        I = nd.sigmoid(nd.dot(X, W_xi) + nd.dot(H, W_hi) + b_i)
        F = nd.sigmoid(nd.dot(X, W_xf) + nd.dot(H, W_hf) + b_f)
        O = nd.sigmoid(nd.dot(X, W_xo) + nd.dot(H, W_ho) + b_o)
        C_tilda = nd.tanh(nd.dot(X, W_xc) + nd.dot(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [9]:
# 训练和创作
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                            vocab_size, ctx, corpus_indices, idx_to_char,
                            char_to_idx, False, num_epochs, num_steps, lr,
                            clipping_theta, batch_size, pred_period, pred_len,
                            prefixes)


epoch 40, perplexity 219.243301, time 10.06 sec
 - 分开 我不的我 我不你的 我不的我 我不的你 我不的我 我不的你 我不的我 我不的你 我不的我 我不的你
 - 不分开 我不的你 我不的我 我不的你 我不的我 我不的你 我不的我 我不的你 我不的我 我不的你 我不的我
epoch 80, perplexity 69.325723, time 10.79 sec
 - 分开 我想想你想你 我不要 我不 我不 我不要 我不不 我不不 我不不 我不不 我不不 我不不 我不不 
 - 不分开 我想想你想想 我不要 我不 我不 我不要 我不不 我不不 我不不 我不不 我不不 我不不 我不不 
epoch 120, perplexity 16.331127, time 9.60 sec
 - 分开 我想你你看经 我想想你 你我的这面 你你去觉 我不要这生 我知知觉 我不了这节 后知后觉 我不了这
 - 不分开 我想要你 我不要 我不要这样 我不去觉 我爱了这节活 后知后觉 我该了这生活 后知后觉 我该了这生
epoch 160, perplexity 4.301546, time 9.62 sec
 - 分开 你说你的你笑就  说 你想了了了我 说散 你想很久了吧? 我不想你远远久幽 想想大你 我我想多 我
 - 不分开 我已经这样奏我的手不能不开 爱能不能够永远不不没 不不要再想 我不能再想 我不 我不 我不能 爱情
